# Receiving
## Example 1. "Hello World" with direct queue messaging.

![Image](https://www.rabbitmq.com/img/tutorials/receiving.png)
Our second program receive.py will receive messages from the queue and print them on the screen.

Again, first we need to connect to RabbitMQ server. The code responsible for connecting to Rabbit is the same as previously.




In [1]:
import pika

connection = pika.BlockingConnection(pika.ConnectionParameters(
        host='192.168.99.100'))
channel = connection.channel()

The next step, just like before, is to make sure that the queue exists. **Creating a queue using *queue_declare* is idempotent ‒ we can run the command as many times as we like, and only one will be created.**


In [2]:
channel.queue_declare(queue='hello')

<METHOD(['channel_number=1', 'frame_type=1', "method=<Queue.DeclareOk(['consumer_count=0', 'message_count=1', 'queue=hello'])>"])>

You may ask why we declare the queue again ‒ we have already declared it in our previous code. We could avoid that if we were sure that the queue already exists. For example if send.py program was run before. But we're not yet sure which program to run first. In such cases it's a good practice to repeat declaring the queue in both programs.



Receiving messages from the queue is more complex. It works by subscribing a callback function to a queue. Whenever we receive a message, this callback function is called by the Pika library. In our case this function will print on the screen the contents of the message.



In [3]:
def callback(ch, method, properties, body):
    print " [x] Received %r" % (body,)

Next, we need to tell RabbitMQ that this particular callback function should receive messages from our **hello** queue:



In [4]:
channel.basic_consume(callback,
                      queue='hello',
                      no_ack=True)

'ctag1.e25fc417d8184940b6d0f3f2303bf850'

For that command to succeed we must be sure that a queue which we want to subscribe to exists. Fortunately we're confident about that ‒ we've created a queue above ‒ using queue_declare.

The *no_ack* parameter will be described **later on**.

And finally, we enter a never-ending loop that waits for data and runs callbacks whenever necessary.

In [5]:
print ' [*] Waiting for messages. To exit press CTRL+C'
channel.start_consuming()


 [*] Waiting for messages. To exit press CTRL+C
 [x] Received 'Hello Worl22d!'
 [x] Received 'Hello Worl22d!'
 [x] Received 'Hello Worl22d!'
 [x] Received 'Hello Worl22d!'


KeyboardInterrupt: 

Before exiting the program we need to make sure the network buffers were flushed and our message was actually delivered to RabbitMQ. We can do it by gently closing the connection.



In [6]:
connection.close()